In [1]:
from download_delgado.delgado_datasets import DownloadAndConvertDelgadoDatasets
from mleap.data import Data
from mleap.data import ModelsContainer
from mleap.experiments import TestOrchestrator
from mleap.analyze_results import AnalyseResults

### Step 1: Load the datasets

In [2]:
#load datasets as pandas DataFrame
#each dataset needs to have metadata attached to it containing the following keys:
# class_name: (string) name of column containing the column name in which the labels are stored
# dataset_name: (string) name of the dataset
delgado = DownloadAndConvertDelgadoDatasets()
datasets, metadata = delgado.download_and_extract_datasets(verbose = False)

Error: Dataset Delgado_data/molec-biol-protein-second has a different number of arff files


### Step 2: Save the datasets in HDF5 database

In [3]:
data = Data()
data.pandas_to_db(save_loc_hdf5='delgado_datasets/', datasets=datasets, 
                  dts_metadata=metadata, save_loc_hdd='data/delgado.hdf5')

### Stp 3: Instantiate models and Test Orchestrator

In [4]:
models_container = ModelsContainer()
instantiated_models = models_container.instantiate_models(verbose=True, 
                                                          num_parallel_jobs=4, 
                                                          RandomForestClassifier=None, 
                                                          SVM=None, 
                                                          LogisticRegression=None)

test_o = TestOrchestrator(hdf5_input_path='data/delgado.hdf5', hdf5_output_path='data/experiments.hdf5')

### Step 4: Split datasets

In [5]:
dts_list = data.list_datasets(hdf5_loc='data/delgado.hdf5',hdf5_group='delgado_datasets/')
split_dts_list = test_o.split_datasets(dataset_paths=dts_list, split_datasets_dir='split_datasets/')

### Step 5: Run the experiments

In [6]:
test_o.run_experiments(split_dts_list[0:3], instantiated_models)

*** Training models on dataset: abalone. Total datasets processed: 0/3 ***
Fitting 3 folds for each of 36 candidates, totalling 108 fits
Fitting 3 folds for each of 4 candidates, totalling 12 fits


[Parallel(n_jobs=4)]: Done 108 out of 108 | elapsed:    4.6s finished
[Parallel(n_jobs=4)]: Done  12 out of  12 | elapsed:    2.1s finished


Fitting 3 folds for each of 2 candidates, totalling 6 fits


[Parallel(n_jobs=4)]: Done   6 out of   6 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=4)]: Done   6 out of   6 | elapsed:    0.1s finished


*** Training models on dataset: acute_inflammation. Total datasets processed: 1/3 ***
Fitting 3 folds for each of 36 candidates, totalling 108 fits
Fitting 3 folds for each of 4 candidates, totalling 12 fits


...

[Parallel(n_jobs=4)]: Done   5 out of  12 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=4)]: Done  12 out of  12 | elapsed:    0.0s finished
[Parallel(n_jobs=4)]: Done   6 out of   6 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=4)]: Done   6 out of   6 | elapsed:    0.0s finished


*** Training models on dataset: acute_nephritis. Total datasets processed: 2/3 ***
Fitting 3 folds for each of 36 candidates, totalling 108 fits
Fitting 3 folds for each of 4 candidates, totalling 12 fits


[Parallel(n_jobs=4)]: Done 108 out of 108 | elapsed:    2.0s finished


Fitting 3 folds for each of 2 candidates, totalling 6 fits


[Parallel(n_jobs=4)]: Done   5 out of  12 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=4)]: Done  12 out of  12 | elapsed:    0.0s finished
[Parallel(n_jobs=4)]: Done   6 out of   6 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=4)]: Done   6 out of   6 | elapsed:    0.0s finished


### Step 6: Analyze the results

In [7]:
analyze = AnalyseResults('data/experiments.hdf5')
t_test, t_test_df = analyze.perform_t_test()
print('******t-test******')
print(t_test_df)

sign_test, sign_test_df = analyze.perform_sign_test()
print('******sign test******')
print(sign_test_df)

t_test_bonferroni, t_test_bonferroni_df = analyze.perform_t_test_with_bonferroni_correction()
print('******t-test bonferroni correction******')
print(t_test_bonferroni_df)
wilcoxon_test, wilcoxon_test_df = analyze.perform_wilcoxon()
print('******Wilcoxon test******')
print(wilcoxon_test_df)

friedman_test, friedman_test_df = analyze.perform_friedman_test()
print('******Friedman test******')
print(friedman_test_df)

******t-test******
                                          pair  t_statistic  p_value
0  LogisticRegression - RandomForestClassifier         2.92     0.35
1                     LogisticRegression - SVM         2.92     0.35
2                 RandomForestClassifier - SVM         2.92     0.35
******sign test******
                                          pair   p_value
0  LogisticRegression - RandomForestClassifier  0.248213
1                     LogisticRegression - SVM  0.248213
2                 RandomForestClassifier - SVM  0.248213
******t-test bonferroni correction******
                                          pair  t_statistic  p_value
0  LogisticRegression - RandomForestClassifier         5.34     0.35
1                     LogisticRegression - SVM         5.34     0.35
2                 RandomForestClassifier - SVM         5.34     0.35
******Wilcoxon test******
                                          pair  statistic   p_value
0  LogisticRegression - RandomForestClassifi

/media/viktor/Data/PhD/ml_experiments/py36/lib/python3.6/site-packages/scipy/stats/morestats.py:2385: UserWarning: Warning: sample size too small for normal approximation.
  warnings.warn("Warning: sample size too small for normal approximation.")
